In [4]:
!pip install -r requeriments.txt

In [1]:
import tenseal as ts
import os
import pickle
import numpy as np

In [80]:
context = ts.context(
    ts.SCHEME_TYPE.CKKS,   # Esquema CKKS para valores de ponto flutuante
    8192,                  # poly_modulus_degree = 4096
    None,                    # Um valor de padrão para a cadeia de módulos
    [40, 21, 21, 40]       # Coefficient modulus bits sizes (adequado para CKKS de 4096)
)

In [5]:
def context():
    """
    This function is used to create the context of the homomorphic encryption:
    it is used to create the keys and the parameters of the encryption scheme (CKKS).

    :return: the context of the homomorphic encryption
    """
    cont = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        # This means that the coefficient modulus will contain 4 primes of 60 bits, 40 bits, 40 bits, and 60 bits.
        # coeff_mod_bit_sizes=[60, 40, 40, 60]
        coeff_mod_bit_sizes=[40, 21, 21, 40]
    )

    #cont.generate_galois_keys()  # You can create the Galois keys by calling generate_galois_keys
    cont.generate_relin_keys()
    cont.global_scale = 2 ** 30  # global_scale: the scaling factor, here set to 2**40 (same that pow(2, 40))
    return cont

In [6]:
def read_query(file_path):
    """
    This function is used to read a pickle file.

    :param file_path: the path of the file to read
    :return: the query and the context
    """
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            """
            # pickle.load(f)  # load to read file object

            file_str = f.read()
            client_query1 = pickle.loads(file_str)  # loads to read str class
            """
            query_str = pickle.load(file)

        contexte = query_str["context"]  # ts.context_from(query["contexte"])
        del query_str["context"]
        return query_str, contexte

    else:
        print("The file doesn't exist")

In [7]:
def combo_keys(client_path="secret.pkl", server_path="server_key.pkl"):
    """
    To create the public/private keys combination
    args:
        client_path: path to save the secret key (str)
        server_path: path to save the server public key (str)
    """
    context_client = context()
    write_query(f'context/{client_path}', {"context": context_client.serialize(save_secret_key=True)})
    write_query(f'context/{server_path}', {"context": context_client.serialize()})

    _, context_client = read_query(f'context/{client_path}')
    _, context_server = read_query(f'context/{server_path}')

    context_client = ts.context_from(context_client)
    context_server = ts.context_from(context_server)
    print("Is the client context private?", ("Yes" if context_client.is_private() else "No"))
    print("Is the server context private?", ("Yes" if context_server.is_private() else "No"))

In [8]:
def write_query(file_path, client_query):
    """
    This function is used to write a pickle file.

    :param file_path: the path of the file to write

    :param client_query: the query to write
    """
    with open(file_path, 'wb') as file:  # 'ab' to add existing file
        encode_str = pickle.dumps(client_query)
        file.write(encode_str)

In [9]:
combo_keys()

Is the client context private? Yes
Is the server context private? No


In [86]:
with open('secret.pkl', 'rb') as file:
    secret = pickle.load(file)

context_client = ts.context_from(secret["context"])

In [87]:
with open('server_key.pkl', 'rb') as file:
    secret = pickle.load(file)

context_server = ts.context_from(secret["context"])

In [40]:
context_client.secret_key()

In [4]:
import tensorflow as tf

model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(10, 10, 1)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(64,  activation='relu'),
            tf.keras.layers.Dense(32,  activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax'),

        ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-09-21 18:55:57.864688: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-21 18:56:00.432440: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-21 18:56:00.442608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 18:56:07.028596: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-21 18:56:26.719215: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-21 18:56:26.720674: W tensorflow/core/common_r

In [5]:
flat_model = []
pesos      = model.get_weights()

for p in pesos:
    flat_model.extend(p.flatten())


In [88]:
he_layer = ts.ckks_tensor(context_client, [1, 2, 3, 4])

In [89]:
a = ts.ckks_tensor(context_client,[1/10])

he_layer *= a


In [8]:
a = he_layer.serialize()

In [9]:
new_he = ts.ckks_tensor_from(context_server, a)
new_he += 100
b = new_he.serialize()

In [10]:
resp = ts.ckks_tensor_from(context_client, b)
reshaped_model  = []
decrypted_model = resp.decrypt().raw

for p in pesos:
    reshaped_model.append(np.reshape(decrypted_model[:p.size], p.shape))
    decrypted_model = decrypted_model[p.size:]

In [11]:
model.set_weights(reshaped_model)

In [12]:
model.get_weights()

[array([[200.08612, 200.02266, 199.82983, 200.21436, 199.8971 , 200.01328,
         200.12451, 199.82745, 199.90315, 200.05154],
        [199.83098, 200.18414, 199.8082 , 199.78523, 200.21219, 200.23267,
         200.20174, 199.81117, 200.05728, 200.2092 ],
        [200.00955, 199.9493 , 199.80568, 200.13058, 199.82507, 199.97203,
         199.972  , 200.15419, 200.04436, 200.09207],
        [200.07076, 199.79007, 200.03278, 200.09113, 199.88052, 199.99623,
         199.92026, 200.06418, 199.98376, 199.78018],
        [199.80911, 199.84721, 199.77411, 200.09828, 199.79605, 199.97661,
         199.88982, 199.86215, 200.07483, 200.04381],
        [200.07211, 200.00148, 200.18217, 199.99603, 199.78609, 200.16315,
         199.82014, 199.85194, 200.07477, 200.12158],
        [199.80658, 200.14082, 199.85971, 200.14185, 199.85043, 200.04839,
         200.19328, 200.017  , 199.7731 , 199.8791 ],
        [200.06915, 200.12115, 199.82039, 200.0864 , 199.82788, 199.8415 ,
         200.22272, 19